In [2]:
import yfinance as yf
from openbb import obb
import pandas as pd
import xlwings as xw
import numpy as np
from collections import defaultdict

In [3]:
input_path = r"input\Securities List_AMC 2025.xlsx"
output_path_isin_ticker = r"output\isin_to_ticker.xlsx"
output_path_metadata = r"output\full_metadata.xlsx"

In [4]:
df_securities_spx = pd.read_excel(input_path,
                                    sheet_name="SPX",
                                    usecols="A, C, D",
                                    names=['name', 'currency', 'ISIN'])
df_securities_sxxp = pd.read_excel(input_path,
                                    sheet_name="SXXP",
                                    usecols="A, C, D",
                                    names=['name', 'currency', 'ISIN'])

df_securities = pd.concat([df_securities_spx, df_securities_sxxp], axis=0)
df_securities.replace(0, pd.NA, inplace=True)
df_securities = df_securities.loc[-df_securities.ISIN.isna()]
df_securities.head()

,name,currency,ISIN
1,Accenture PLC,USD,IE00B4BNMY34
2,Adobe Inc,USD,US00724F1012
3,Advanced Micro Devices Inc,USD,US0079031078
4,Akamai Technologies Inc,USD,US00971T1016
5,Amphenol Corp,USD,US0320951017


In [5]:
test_isin = "IE00B4BNMY34"
yf.set_config()
test_data = yf.Ticker(test_isin)
test_data.ticker

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


'ACN'

In [6]:
dict_isin_tickers = defaultdict(list)
# meta_records = []
for name, sec_id in zip(df_securities.name, df_securities.ISIN):
    try:
        t = yf.Ticker(sec_id)
        dict_isin_tickers["ISIN"].append(sec_id)
        dict_isin_tickers["ticker"].append(t.ticker)

        # data = pd.Series(yf.Ticker(sec_id).fast_info)
        # info = t.info  # slower, but includes country/sector/industry
        # meta_records.append({
        #     "ticker": info.ticker,
        #     "country": info.get("country"),
        #     "sector": info.get("sector"),
        #     "industry": info.get("industry"),
        #     "currency": info.get("currency"),
        #     "marketCap": info.get("marketCap"),
        #     "beta": info.get("beta"),
        #     "exchange": info.get("exchange"),
        # })
    except ValueError as e:
        dict_isin_tickers["ISIN"].append(sec_id)
        dict_isin_tickers["ticker"].append(pd.NA)
        print(e, name)
# metadata = pd.DataFrame(meta_records).set_index("ticker")

Invalid ISIN number: US03662Q1058 ANSYS Inc
Invalid ISIN number: US48203R1041 Juniper Networks Inc
Invalid ISIN number: US2547091080 Discover Financial Services
Invalid ISIN number: US92556H2067 Paramount Global
Invalid ISIN number: US9314271084 Walgreens Boots Alliance Inc
Invalid ISIN number: US42809H1077 Hess Corp
Invalid ISIN number: FI0009000202 Kesko Oyj
Invalid ISIN number: GB00BY9D0Y18 Direct Line Insurance Group PLC
Invalid ISIN number: GB00B1VZ0M25 Hargreaves Lansdown PLC
Invalid ISIN number: GB00BDVZYZ77 International Distribution Services PLC
Invalid ISIN number: NO0003043309 Kongsberg Gruppen ASA
Invalid ISIN number: SE0017161243 Fortnox AB
Invalid ISIN number: SE0005127818 Sagax AB
Invalid ISIN number: FR0011675362 Neoen SA


In [7]:
df_isin_ticker = pd.DataFrame(dict_isin_tickers)
# df_isin_ticker.to_excel(output_path_isin_ticker, sheet_name='isin_ticker', index=False)


In [12]:
isin_issues = df_isin_ticker.loc[df_isin_ticker.ticker.isna(), "ISIN"].to_list()
isin_issues

['US03662Q1058',
 'US48203R1041',
 'US2547091080',
 'US92556H2067',
 'US9314271084',
 'US42809H1077',
 'FI0009000202',
 'GB00BY9D0Y18',
 'GB00B1VZ0M25',
 'GB00BDVZYZ77',
 'NO0003043309',
 'SE0017161243',
 'SE0005127818',
 'FR0011675362']

In [9]:
df_with_ticker = pd.merge(df_securities, df_isin_ticker, on='ISIN')
df_with_ticker.head()

,name,currency,ISIN,ticker
0,Accenture PLC,USD,IE00B4BNMY34,ACN
1,Adobe Inc,USD,US00724F1012,ADBE
2,Advanced Micro Devices Inc,USD,US0079031078,AMD
3,Akamai Technologies Inc,USD,US00971T1016,AKAM
4,Amphenol Corp,USD,US0320951017,APH


In [10]:
BATCH_SIZE = 20
results = []
tickers = df_with_ticker.ticker.dropna()
for i in range(0, len(tickers), BATCH_SIZE):
    batch = tickers[i:i+BATCH_SIZE]
    df = obb.equity.profile(batch, provider='yfinance').to_df()
    results.append(df)
    print(i/len(tickers)*100, '%')

0.0 %
1.8365472910927456 %
3.6730945821854912 %
5.5096418732782375 %
7.3461891643709825 %
9.182736455463727 %
11.019283746556475 %
12.85583103764922 %
14.692378328741965 %
16.528925619834713 %
18.365472910927455 %
20.2020202020202 %
22.03856749311295 %
23.875114784205692 %
25.71166207529844 %
27.548209366391184 %
29.38475665748393 %
31.22130394857668 %
33.057851239669425 %
34.89439853076217 %
36.73094582185491 %
38.56749311294766 %
40.4040404040404 %
42.240587695133144 %
44.0771349862259 %
45.91368227731864 %
47.750229568411385 %
49.586776859504134 %
51.42332415059688 %
53.25987144168962 %
55.09641873278237 %


500 Server Error:  for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PUIG.MC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PUIG.MC&crumb=Xlk1BqPOKBY


56.93296602387512 %
58.76951331496786 %
60.60606060606061 %
62.44260789715336 %
64.2791551882461 %
66.11570247933885 %
67.95224977043158 %
69.78879706152433 %
71.62534435261708 %
73.46189164370982 %
75.29843893480258 %
77.13498622589532 %
78.97153351698806 %
80.8080808080808 %
82.64462809917356 %
84.48117539026629 %
86.31772268135904 %
88.1542699724518 %
89.99081726354453 %
91.82736455463728 %
93.66391184573003 %
95.50045913682277 %
97.33700642791551 %
99.17355371900827 %


In [ ]:
df_results = pd.concat(results, ignore_index=True)
# df_results.to_excel(output_path_metadata)